In [ ]:
! uv pip install langchain groq tiktoken rapidocr-onnxruntime python-dotenv langchain-community sentence-transformers faiss-cpu langchain-groq

Using Python 3.11.9 environment at: G:\Data Science\MLOPs\LLMOPS series\.venv
Audited 8 packages in 8.00s


In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

if groq_api_key is None:
	raise ValueError("GROQ_API_KEY not found in environment variables. Please check your .env file.")

os.environ["GROQ_API_KEY"] = groq_api_key

### Data Ingestion

In [ ]:
from langchain.document_loaders import TextLoader

loader = TextLoader("../data/sample_text.txt", encoding="utf8")
documents = loader.load()

In [25]:
documents[0].page_content[ : 100]

'Neural Radiance Fields (NeRF) represent a cutting-edge technique in 3D computer graphics that uses n'

In [26]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap = 20
)

text_chunks = text_splitter.split_documents(documents)
text_chunks

[Document(metadata={'source': '../data/sample_text.txt'}, page_content='Neural Radiance Fields (NeRF) represent a cutting-edge technique in 3D computer graphics that uses neural networks to generate novel views of complex 3D scenes. Unlike traditional graphics pipelines'),
 Document(metadata={'source': '../data/sample_text.txt'}, page_content='graphics pipelines that rely on explicit geometry and texture maps, NeRF models learn an implicit volumetric representation of a scene by optimizing a continuous function that maps spatial'),
 Document(metadata={'source': '../data/sample_text.txt'}, page_content='that maps spatial coordinates and viewing directions to color and volume density. This function is parameterized by a multilayer perceptron (MLP), allowing the scene to be rendered with'),
 Document(metadata={'source': '../data/sample_text.txt'}, page_content='to be rendered with photorealistic detail and consistent lighting effects.'),
 Document(metadata={'source': '../data/sample_text.

In [34]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# Load the model
embedding_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

C:\Users\Hamza\AppData\Local\Temp\ipykernel_1964\3781062547.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [38]:
import numpy as np

# Example texts
texts = [
    "Hello, how are you?",
    "This is a sample sentence for embeddings.",
    "RAG applications are very powerful."
]

# Generate embeddings using embed_documents
embeddings = embedding_model.embed_documents(texts)
embeddings = np.array(embeddings)
print("Embeddings shape:", embeddings.shape)

Embeddings shape: (3, 384)


In [39]:
vectorstore = FAISS.from_documents(text_chunks, embedding_model)

In [41]:
vectorstore

In [ ]:
retriever = vectorstore.as_retriever()

# Perform similarity search
query = "What is the main difference between NeRF and traditional 3D graphics pipelines?"
#query = "How does NeRF use volumetric rendering to produce photorealistic images?"
#query = "What techniques have been developed to speed up NeRF training and inference?"

docs = vectorstore.similarity_search(query, k=4)

In [47]:
# Display the results
for i, doc in enumerate(docs):
    print(f"Document {i+1}:")
    print(doc.page_content)
    print("-" * 200)

Document 1:
In the context of 3D graphics pipelines, integrating NeRF with traditional graphics tools poses challenges, such as converting neural representations into mesh-based formats usable in real-time
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Document 2:
graphics pipelines that rely on explicit geometry and texture maps, NeRF models learn an implicit volumetric representation of a scene by optimizing a continuous function that maps spatial
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Document 3:
One of the core innovations in NeRF is the use of volumetric rendering, where a ray is cast from the camera into the scene and sampled at multiple depths. At each sample point, th

In [48]:
from langchain.prompts import ChatPromptTemplate

prompt_template = """
You are an AI assistant for question-answering tasks. 
Use the retrieved context to answer the question as accurately as possible. 
If the answer is not in the context, respond with "I don't know." 
Keep your answer concise and no longer than ten sentences.

Question: {question}
Context: {context}
Answer:
"""

In [49]:
prompt = ChatPromptTemplate.from_template(prompt_template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nYou are an AI assistant for question-answering tasks. \nUse the retrieved context to answer the question as accurately as possible. \nIf the answer is not in the context, respond with "I don\'t know." \nKeep your answer concise and no longer than ten sentences.\n\nQuestion: {question}\nContext: {context}\nAnswer:\n'), additional_kwargs={})])

In [53]:
from langchain.schema.output_parser import StrOutputParser
from langchain_groq import ChatGroq

output_parser = StrOutputParser()

llm_model = ChatGroq(model_name="openai/gpt-oss-20b")

In [54]:
from langchain.schema.runnable import RunnablePassthrough

rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm_model
    | output_parser
)

In [55]:
rag_chain.invoke("How does NeRF use volumetric rendering to produce photorealistic images?")
#rag_chain.invoke("What techniques have been developed to speed up NeRF training and inference?")

'NeRF uses volumetric rendering by casting a ray from the camera into the scene and sampling many points along that ray.  \nAt each sampled depth the network (an MLP) predicts a density (volume density) and an RGB color for that point.  \nThese predictions are treated as a continuous radiance field that represents the scene’s geometry and appearance implicitly.  \nThe algorithm then integrates the predicted densities and colors along the ray using the volume rendering equation, which accumulates color while attenuating light according to the density.  \nThe integrated result for each pixel yields a photorealistic image.  \nTraining optimizes the MLP to minimize the difference between these rendered images and a set of input photographs taken from known camera poses.  \nThus NeRF learns a continuous volumetric representation that can generate novel views with high realism.'

In [56]:
! uv pip install structlog

Using Python 3.11.9 environment at: G:\Data Science\MLOPs\LLMOPS series\.venv
Resolved 1 package in 99ms
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 1 package in 86ms
 + structlog==25.4.0
